In [60]:
from sklearn.ensemble import GradientBoostingClassifier
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
import joblib
import json
import glob
import os

In [61]:
def encoder(data):
    unique_values = {}
    for d in data:
        if isinstance(data[d][0], str):
            values_list = list(data[d].unique())
            temp_dict = {}
            for item in values_list:
                index = values_list.index(item)
                temp_dict.update({item:index})
            unique_values.update({d:temp_dict})
    for i in data.columns:
        if isinstance(data[i][0], str):
            column = data[i]
            for d in range(len(column)):
                column[d] = unique_values[i][column[d]]
    return data, unique_values

def prediction_encoder(data,unique_values):
    for i in data.columns:
        if isinstance(data[i][0], str):
            column = data[i]
            for d in range(len(column)):
                column[d] = unique_values[i][column[d]]
    return data

def get_prediction(data, model):
    y_pred = model.predict(data)
    y_pred = y_pred.tolist()
    return y_pred[0]

In [62]:
data = pd.read_csv("train.csv")
data = data.dropna()
data.head()

,ID,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,...,gender,ethnicity,jaundice,austim,contry_of_res,used_app_before,result,age_desc,relation,Class/ASD
0,1,1,0,1,1,1,1,0,1,1,...,f,White-European,no,no,United States,no,7.819715,18 and more,Self,0
1,2,0,0,0,0,0,0,0,0,0,...,f,South Asian,no,no,Australia,no,10.544296,18 and more,?,0
2,3,1,1,1,1,1,1,0,0,1,...,f,White-European,no,no,United Kingdom,no,13.167506,18 and more,Self,1
3,4,0,0,0,1,0,0,0,0,0,...,f,South Asian,no,no,New Zealand,no,1.530098,18 and more,?,0
4,5,0,0,0,0,1,0,0,0,1,...,m,Black,no,yes,Italy,no,7.949723,18 and more,Self,0


In [63]:
print(f"The length of dataset :{len(data)}")
data['ethnicity'] = data['ethnicity'].replace('?', 'Others')
data['ethnicity'] = data['ethnicity'].replace('others', 'Others')
data['relation'] = data['relation'].replace('?', 'Others')
data = data.apply(lambda x: x.str.lower() if x.dtype == 'object' else x)

The length of dataset :800


In [64]:
data.head()

,ID,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,...,gender,ethnicity,jaundice,austim,contry_of_res,used_app_before,result,age_desc,relation,Class/ASD
0,1,1,0,1,1,1,1,0,1,1,...,f,white-european,no,no,united states,no,7.819715,18 and more,self,0
1,2,0,0,0,0,0,0,0,0,0,...,f,south asian,no,no,australia,no,10.544296,18 and more,others,0
2,3,1,1,1,1,1,1,0,0,1,...,f,white-european,no,no,united kingdom,no,13.167506,18 and more,self,1
3,4,0,0,0,1,0,0,0,0,0,...,f,south asian,no,no,new zealand,no,1.530098,18 and more,others,0
4,5,0,0,0,0,1,0,0,0,1,...,m,black,no,yes,italy,no,7.949723,18 and more,self,0


In [65]:
data_u = data.copy()
updated_data, unique_values = encoder(data_u)
with open("unique_values.json", "w") as outfile:
    json.dump(unique_values, outfile)
updated_data.to_csv("temp.csv",index=False)
updated_data= pd.read_csv("temp.csv")
##load unique values map
with open("unique_values.json", 'r') as openfile:
    unique_values = json.load(openfile)
    
## load feature map
with open("map.json", 'r') as openfile:
    map_values = json.load(openfile)
map_values = dict(map_values)

C:\Users\Mehedi Tamim\AppData\Local\Temp\ipykernel_3492\2477670212.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  column[d] = unique_values[i][column[d]]
C:\Users\Mehedi Tamim\AppData\Local\Temp\ipykernel_3492\2477670212.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  column[d] = unique_values[i][column[d]]
C:\Users\Mehedi Tamim\AppData\Local\Temp\ipykernel_3492\2477670212.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 

In [66]:
X = updated_data.drop(['Class/ASD','ID'], axis=1)
y = updated_data['Class/ASD']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [67]:
model = RandomForestClassifier()
model.fit(X_train, y_train)
model.score(X_train, y_train)

1.0

In [82]:
data = {
    "A1_Score":1,
    "A2_Score":1,
    "A3_Score":1,
    "A4_Score":1,
    "A5_Score":1,
    "A6_Score":1,
    "A7_Score":1,
    "A8_Score":1,
    "A9_Score":1,
    "A10_Score":0,
    "age": 20.30594341,
    "gender":"f",
    "ethnicity":"white-european",
    "jaundice":"yes",
    "austim": "no",
    "contry_of_res": "United Kingdom",
    "used_app_before": "no",
    "result": 11.90246137,
    "age_desc": "18 and more",
    "relation": "Self" 
}
data = {key: str(value).lower() if isinstance(value, str) else value for key, value in data.items()}

text = ""
count = 0
input_list = pd.DataFrame(data, index=[1])
input_list = input_list.reset_index(drop=True)
data = prediction_encoder(input_list,unique_values)
if get_prediction(data, model) == 0:
    print(f"\n Austism Disorder---> Negative \n")
elif get_prediction(data, model) == 1:
    print(f"\n Austism Disorder---> Positive \n")

    for key, value in data.items():
        if key in map_values.keys():
            if value.values[0] ==1:
                count += 1
                text = text + f"\n {count}. {map_values[key]} \n"
     
print(f"""
      
===============================================================
positive reaction when they got 
    
    {text}
===============================================================
    
    
    """)


 Austism Disorder---> Positive 


      
positive reaction when they got 
    
    
 1. things are calm and quite 

 2. People Acknowledge their perspective, use visual  aids like diagram or chart etc and give me contextetual information 

 3. Allow to do multitasking and help them to complete their work 

 4. Provide minimum interruption and encourage for time management to do the task 

 5. people express their thought and intention clearly , actively listening 

 6. Engaging in active conversion , use non verbal communication and show interest in their conversation 

 7. Use visual aids , summarize complex plots and discuss and share their tought 

 8. Encourage their hobbies and show interest in their findings 

 9. Encourage open communication verbally instead of facial expression 

    
    
    


C:\Users\Mehedi Tamim\AppData\Local\Temp\ipykernel_3492\2477670212.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  column[d] = unique_values[i][column[d]]
C:\Users\Mehedi Tamim\AppData\Local\Temp\ipykernel_3492\2477670212.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  column[d] = unique_values[i][column[d]]
C:\Users\Mehedi Tamim\AppData\Local\Temp\ipykernel_3492\2477670212.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 

In [83]:
model = SVC()
model.fit(X_train, y_train)
model.score(X_train, y_train)

0.85625

In [84]:
data = {
    "A1_Score":1,
    "A2_Score":1,
    "A3_Score":1,
    "A4_Score":1,
    "A5_Score":1,
    "A6_Score":1,
    "A7_Score":1,
    "A8_Score":1,
    "A9_Score":1,
    "A10_Score":0,
    "age": 20.30594341,
    "gender":"f",
    "ethnicity":"white-european",
    "jaundice":"yes",
    "austim": "no",
    "contry_of_res": "United Kingdom",
    "used_app_before": "no",
    "result": 11.90246137,
    "age_desc": "18 and more",
    "relation": "Self" 
}
data = {key: str(value).lower() if isinstance(value, str) else value for key, value in data.items()}

text = ""
count = 0
input_list = pd.DataFrame(data, index=[1])
input_list = input_list.reset_index(drop=True)
data = prediction_encoder(input_list,unique_values)
if get_prediction(data, model) == 0:
    print(f"\n Austism Disorder---> Negative \n")
elif get_prediction(data, model) == 1:
    print(f"\n Austism Disorder---> Positive \n")

    for key, value in data.items():
        if key in map_values.keys():
            if value.values[0] ==1:
                count += 1
                text = text + f"\n {count}. {map_values[key]} \n"
     
print(f"""
      
===============================================================
positive reaction when they got 
    
    {text}
===============================================================
    
    
    """)


 Austism Disorder---> Positive 


      
positive reaction when they got 
    
    
 1. things are calm and quite 

 2. People Acknowledge their perspective, use visual  aids like diagram or chart etc and give me contextetual information 

 3. Allow to do multitasking and help them to complete their work 

 4. Provide minimum interruption and encourage for time management to do the task 

 5. people express their thought and intention clearly , actively listening 

 6. Engaging in active conversion , use non verbal communication and show interest in their conversation 

 7. Use visual aids , summarize complex plots and discuss and share their tought 

 8. Encourage their hobbies and show interest in their findings 

 9. Encourage open communication verbally instead of facial expression 

    
    
    


C:\Users\Mehedi Tamim\AppData\Local\Temp\ipykernel_3492\2477670212.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  column[d] = unique_values[i][column[d]]
C:\Users\Mehedi Tamim\AppData\Local\Temp\ipykernel_3492\2477670212.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  column[d] = unique_values[i][column[d]]
C:\Users\Mehedi Tamim\AppData\Local\Temp\ipykernel_3492\2477670212.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 

In [85]:
model = GradientBoostingClassifier()
model.fit(X_train, y_train)
model.score(X_train, y_train)

0.9609375

In [86]:
data = {
    "A1_Score":1,
    "A2_Score":1,
    "A3_Score":1,
    "A4_Score":1,
    "A5_Score":1,
    "A6_Score":1,
    "A7_Score":1,
    "A8_Score":1,
    "A9_Score":1,
    "A10_Score":0,
    "age": 20.30594341,
    "gender":"f",
    "ethnicity":"white-european",
    "jaundice":"yes",
    "austim": "no",
    "contry_of_res": "United Kingdom",
    "used_app_before": "no",
    "result": 11.90246137,
    "age_desc": "18 and more",
    "relation": "Self" 
}
data = {key: str(value).lower() if isinstance(value, str) else value for key, value in data.items()}

text = ""
count = 0
input_list = pd.DataFrame(data, index=[1])
input_list = input_list.reset_index(drop=True)
data = prediction_encoder(input_list,unique_values)
if get_prediction(data, model) == 0:
    print(f"\n Austism Disorder---> Negative \n")
elif get_prediction(data, model) == 1:
    print(f"\n Austism Disorder---> Positive \n")

    for key, value in data.items():
        if key in map_values.keys():
            if value.values[0] ==1:
                count += 1
                text = text + f"\n {count}. {map_values[key]} \n"
     
print(f"""
      
===============================================================
positive reaction when they got 
    
    {text}
===============================================================
    
    
    """)


 Austism Disorder---> Positive 


      
positive reaction when they got 
    
    
 1. things are calm and quite 

 2. People Acknowledge their perspective, use visual  aids like diagram or chart etc and give me contextetual information 

 3. Allow to do multitasking and help them to complete their work 

 4. Provide minimum interruption and encourage for time management to do the task 

 5. people express their thought and intention clearly , actively listening 

 6. Engaging in active conversion , use non verbal communication and show interest in their conversation 

 7. Use visual aids , summarize complex plots and discuss and share their tought 

 8. Encourage their hobbies and show interest in their findings 

 9. Encourage open communication verbally instead of facial expression 

    
    
    


C:\Users\Mehedi Tamim\AppData\Local\Temp\ipykernel_3492\2477670212.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  column[d] = unique_values[i][column[d]]
C:\Users\Mehedi Tamim\AppData\Local\Temp\ipykernel_3492\2477670212.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  column[d] = unique_values[i][column[d]]
C:\Users\Mehedi Tamim\AppData\Local\Temp\ipykernel_3492\2477670212.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 